## 02 · Build DataLoader from Ray Data  

Instead of using PyTorch’s `DataLoader`, we now build a loader from a **Ray Dataset shard**.  

- `ray.train.get_dataset_shard("train")` → retrieves the shard of the training dataset assigned to the current worker.  
- `.iter_torch_batches()` → streams the shard as PyTorch-compatible batches.  
  * Each batch is a **dictionary** (e.g., `{"image": tensor, "label": tensor}`).  
  * Supports options like `batch_size` and `prefetch_batches` for performance tuning.  

This integration ensures that data is **sharded, shuffled, and moved to the right device automatically**, while still looking and feeling like a familiar PyTorch data loader.  

**Note:** Use [`iter_torch_batches`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.iter_torch_batches.html) to build a PyTorch-compatible data loader from a Ray Dataset. 

In [ ]:
# 02. Build a Ray Data–backed data loader

def build_data_loader_ray_train_ray_data(batch_size: int, prefetch_batches: int = 2):

    # Different: instead of creating a PyTorch DataLoader,
    # fetch the training dataset shard for this worker
    dataset_iterator = ray.train.get_dataset_shard("train")

    # Convert the shard into a PyTorch-style iterator
    # - Returns dict batches: {"image": ..., "label": ...}
    # - prefetch_batches controls pipeline buffering
    data_loader = dataset_iterator.iter_torch_batches(
        batch_size=batch_size, prefetch_batches=prefetch_batches
    )
    
    return data_loader